In [5]:
import numpy as np


class MLP:
    def __init__(self, layers, learning_rate=0.01):
        self.layers = layers
        self.learning_rate = learning_rate
        self.weights = []
        self.biases = []
        self.initialize_weights()

    def initialize_weights(self):
        for i in range(len(self.layers) - 1):
            weight = np.random.randn(self.layers[i], self.layers[i + 1]) * 0.01
            bias = np.zeros((1, self.layers[i + 1]))
            self.weights.append(weight)
            self.biases.append(bias)

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoid_derivative(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))

    def forward_propagation(self, X):
        activations = [X]
        z_values = []

        for i in range(len(self.weights)):
            z = activations[-1] @ self.weights[i] + self.biases[i]
            z_values.append(z)
            activation = self.sigmoid(z)
            activations.append(activation)

        return activations, z_values

    def backward_propagation(self, X, y, activations, z_values):
        m = X.shape[0]
        gradients_w = []
        gradients_b = []
        delta = activations[-1] - y
        for i in reversed(range(len(self.weights))):
            dw = (activations[i].T @ delta) / m
            db = np.sum(delta, axis=0, keepdims=True) / m
            gradients_w.insert(0, dw)
            gradients_b.insert(0, db)

            if i != 0:
                delta = (delta @ self.weights[i].T) * self.sigmoid_derivative(
                    z_values[i - 1]
                )

        return gradients_w, gradients_b

    def update_parameters(self, gradients_w, gradients_b):
        for i in range(len(self.weights)):
            self.weights[i] -= self.learning_rate * gradients_w[i]
            self.biases[i] -= self.learning_rate * gradients_b[i]

    def train(self, X, y, epochs, batch_size=32):
        for epoch in range(epochs):
            for i in range(0, X.shape[0], batch_size):
                X_batch = X[i : i + batch_size]
                y_batch = y[i : i + batch_size]

                activations, z_values = self.forward_propagation(X_batch)
                gradients_w, gradients_b = self.backward_propagation(
                    X_batch, y_batch, activations, z_values
                )
                self.update_parameters(gradients_w, gradients_b)

    def predict(self, X):
        activations, _ = self.forward_propagation(X)
        return activations[-1]

In [6]:
from sklearn.model_selection import KFold


def k_fold_cross_validation(X, y, model_fn, k=5, epochs=100, batch_size=32):
    kf = KFold(n_splits=k)
    accuracies = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        model = model_fn()
        model.train(X_train, y_train, epochs, batch_size)
        predictions = model.predict(X_val)

        accuracy = np.mean((predictions > 0.5) == y_val)
        accuracies.append(accuracy)

    return np.mean(accuracies)

In [7]:
# Generate dataset
np.random.seed(0)
X = np.linspace(-10, 10, 100).reshape(-1, 1)
y = np.sin(X) + 0.1 * np.random.randn(*X.shape)


# Hyperparameter tuning with different configurations
def model_fn():
    return MLP(layers=[1, 10, 10, 1], learning_rate=0.01)


# Perform 5-fold cross-validation
accuracy = k_fold_cross_validation(X, y, model_fn, k=5, epochs=500, batch_size=16)
print("Cross-validated accuracy:", accuracy)

Cross-validated accuracy: 0.0
